### Main goals:
✅ Clarify and reflect on the definition of the term "fake news", which may vary among databases, sometimes non-binary.\
✅ Research, where the data comes from and inspect the data: what are the labels, sources, and authors?\
✅ Is there a person, source or topic which is over- or under-represented?\
✅ Study the literature on how others approach this task. Check the related literature and select your model architecture of choice: LSTM, ...\
✅ Develop a classification model to predict fake news from the text. How do you judge the quality of your results, i.e. which metrics do you consider?
### Optional:
Inspect the falsely classified ones. What can you learn from them?\
Investigate edge cases that you found in your data inspection with respect to how the model learned to identify these.\
Experiment with how you could mitigate if edge cases are covered poorly.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments,Trainer, pipeline
from datasets import load_dataset
import numpy as np
import evaluate
from sklearn.metrics import classification_report
from bertviz import model_view,head_view
import shap
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
from bertopic import BERTopic

## Model Training

#### Downloading the base model and getting the tokenizer
The code below is not necessary to run, as we fine-tuned the model and uploaded it to HuggingFace, therefore just go down to the Model inspection part.

In [ ]:
# model_name = "google-bert/bert-base-uncased"

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

#### Loading the data set

In [ ]:
# data = load_dataset('GonzaloA/fake_news')

In [ ]:
# data = data.remove_columns(['Unnamed: 0','title'])

In [ ]:
# tokenized_data = data.map(tokenize_function, batched=True)

#### Sampling from the data set

In [ ]:
# small_train_dataset = tokenized_data["train"].shuffle(seed=42).select(range(100))
# small_eval_dataset = tokenized_data["validation"].shuffle(seed=42).select(range(100))
# small_test_dataset = tokenized_data['test']

#### Outputting the training arguments

In [ ]:
# training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

#### Loading the evaluation metrics

In [ ]:
# metric = evaluate.load("accuracy")

In [ ]:
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

#### Initializing the Trainer object and fine-tuning the model

In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=small_train_dataset,
#     eval_dataset=small_eval_dataset,
#     compute_metrics=compute_metrics,
# )

In [ ]:
# trainer.train()

#### Getting the predictions

In [ ]:
# test_labels = small_test_dataset['label']
# small_test_dataset = small_test_dataset.remove_columns(['label','token_type_ids'])

In [ ]:
# predictions = trainer.predict(small_test_dataset)

In [ ]:
# predicted_labels = predictions.predictions.argmax(axis=1)

#### Saving the model

In [ ]:
# trainer.save_model('bert-base-uncased-fake-news-classification')

In [ ]:
# tokenizer.save_pretrained('bert-base-uncased-fake-news-classification')

#### Saving the predictions

In [ ]:
# with open('predictions/predictions.txt', 'w') as f:
#     for line in predicted_labels:
#         f.write(f"{line}\n")

## Model Inspection

In [ ]:
load_dotenv()

#### Loading the model

In [ ]:
model_name = 'Flo335/bert-base-uncased-fake-news-classification'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,output_attentions = True,token = os.getenv('huggingface'))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,token = os.getenv('huggingface'))

#### Loading the dataset

In [ ]:
data = load_dataset('GonzaloA/fake_news')

In [ ]:
data = data.remove_columns(['Unnamed: 0','title'])

In [ ]:
with open('predictions/predictions.txt','r') as f:
    predictions = f.read().split('\n')

predictions = [int(i) for i in predictions]

#### Evaluating the predictions

In [ ]:
print(classification_report(data['test']['label'],predictions))

In [ ]:
# count = 0
# for i,j in zip(predictions,data['test']['label']):
#     if i != j:
#         print(count)
#     count += 1

In [ ]:
predictions[59]

In [ ]:
data['test']['label'][59]

#### Exploratory Data Analysis

In [ ]:
count_dict = dict()
for i in data:
    true_news = []
    fake_news = []
    for j in data[i]['label']:
        if j == 0:
            fake_news.append(j)
        else:
            true_news.append(j)
    count_dict[i] = (len(fake_news),len(true_news))

In [ ]:
count_df = pd.DataFrame.from_dict(count_dict,orient = 'index')
count_df.rename(columns = {0:'Fake News',1:'True News'},inplace = True)
count_df.plot.bar(figsize = (10,5));

In [ ]:
len_count_dict = dict()
for i in data:
    true_news = []
    fake_news = []
    for j in data[i]:
        if j['label'] == 0:
            fake_news.append(len(j['text']))
        else:
            true_news.append(len(j['text']))
    len_count_dict[i] = (np.mean(fake_news),np.mean(true_news))

In [ ]:
len_count_df = pd.DataFrame.from_dict(len_count_dict,orient = 'index')
len_count_df.rename(columns = {0:'Fake News',1:'True News'},inplace = True)
len_count_df.plot.bar(figsize = (10,5));

In [ ]:
# tokenizer.get_vocab()

#### Inspecting over-represented tokens in our corpus (BERTopic)

In [ ]:
topic_model_true = BERTopic()
topic_model_fake = BERTopic()

In [ ]:
train = pd.DataFrame(data['train'])

In [ ]:
Fake_news = train[train['label'] == 0]
True_news = train[train['label'] == 1]

In [ ]:
topics_fake, probs_fake = topic_model_fake.fit_transform(Fake_news['text'])

In [ ]:
topics_true, probs_true = topic_model_true.fit_transform(True_news['text'])

In [ ]:
results_bertopic_true = topic_model_true.get_document_info(True_news['text'])

In [ ]:
topic_model_true.get_topic_freq(0)

In [ ]:
topic_model_true.get_topic_info(0)['Representation'][0]

In [ ]:
results_bertopic_fake = topic_model_fake.get_document_info(Fake_news['text'])

In [ ]:
topics_list_true = list(results_bertopic_true[results_bertopic_true['Representative_document'] == True]['Representation'])
topics_list_fake = list(results_bertopic_fake[results_bertopic_fake['Representative_document'] == True]['Representation'])

In [ ]:
topics_list_true = [word.strip() for sublist in topics_list_true for word in sublist]
topics_list_fake = [word.strip() for sublist in topics_list_fake for word in sublist]

In [ ]:
ratio = dict()
for i in Counter(topics_list_fake):
    if i in Counter(topics_list_true):
        ratio[i] = Counter(topics_list_fake)[i]/Counter(topics_list_true)[i]

In [ ]:
sorted_dict = dict(sorted(ratio.items(), key=lambda item: item[1],reverse= True))

In [ ]:
pd.DataFrame.from_dict(sorted_dict,orient = 'index',columns = ['ratio'])[:20].plot.barh();

#### Inspecting over-represented tokens in our corpus (NLTK)

#### Inspecting the model and the head

In [ ]:
inputs = tokenizer.encode(data['test'][9]['text'], return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 

In [ ]:
# head_view(attention, tokens)

In [ ]:
# model_view(attention, tokens)

#### Inspecting the model's attention

In [ ]:
pipe = pipeline('text-classification',model=model_name, top_k=None)

In [ ]:
explainer = shap.Explainer(pipe)

In [ ]:
# 0 - Fake | 1 - True

In [ ]:
# labeled as True (1) but actually Fake (0)
shap_values = explainer(['What???? OMG!!! Elon Musk has been caught making out with Donald Trump. He says he was trying to get more funding for SpaceX.'])

In [ ]:
shap.plots.text(shap_values)